##The R code could a take a while, it is recommended to run it locally.

In [2]:
!git clone https://github.com/davidroad/BMI_7320_Final.git

Cloning into 'BMI_7320_Final'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 49 (delta 20), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (49/49), 765.77 KiB | 3.91 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [3]:
!pwd

/content


In [14]:
!mkdir -p /content/BMI_7320_Final/VEGA_PBMC/save_model
!mkdir -p /content/BMI_7320_Final/VEGA_PBMC/data
##download both h5ad and seurat object




In [15]:
!gdown https://drive.google.com/uc?id=1fWKmDb_3NqjABWzrCU83x0WQMLgZBkkw -O /content/BMI_7320_Final/VEGA_PBMC/data/pbmc_IFN_filtered.Rdata
!gdown https://drive.google.com/uc?id=1Q0he0UJDX90tFxMhwRvf_wXScqh5Aa0V -O /content/BMI_7320_Final/VEGA_PBMC/data/pbmc_IFN_filtered.h5ad

Downloading...
From: https://drive.google.com/uc?id=1fWKmDb_3NqjABWzrCU83x0WQMLgZBkkw
To: /content/BMI_7320_Final/VEGA_PBMC/data/pbmc_IFN_filtered.Rdata
100% 41.8M/41.8M [00:00<00:00, 46.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Q0he0UJDX90tFxMhwRvf_wXScqh5Aa0V
To: /content/BMI_7320_Final/VEGA_PBMC/data/pbmc_IFN_filtered.h5ad
100% 56.4M/56.4M [00:01<00:00, 38.1MB/s]


In [5]:
import sys
sys.path.insert(0,"/content/BMI_7320_Final/VEGA_PBMC/")

In [6]:
 ! pip install scanpy
 ! pip install torch



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 14.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 13.0 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.5-py3-none-any.whl size=86831 sha256=c5f7a4b34000b36897f998e8a769cc4b1b6965b9e50b5254e2da814fe32e10de
  Stored in directory: /root/.cache/pip/wheels/3a/70/07/428d2b58660a1a3b431db59b806a10da736612ebbc66c1bcc5
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8026 sha256=ed11e8e1cfca77260aa3c2802a40c687e60df5814fb63d01dfbed35b27ed6b9b
  Stored in directory: /root/.cache/pip/wheels/6a/aa/b9/eb5d4031476ec10802795b97ccf937b9

In [84]:
import torch
from vega_model import VEGA
from utils import *
from learning_utils import *
import scanpy as sc
from scipy import sparse
from sklearn import preprocessing
import numpy as np
import itertools
import argparse
import os
import pandas as pd
import umap


In [16]:
full_data = sc.read_h5ad("/content/BMI_7320_Final/VEGA_PBMC/data/pbmc_IFN_filtered.h5ad")

In [17]:
print(full_data)


AnnData object with n_obs × n_vars = 13672 × 35635
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'label', 'tsne1', 'tsne2', 'ind', 'stim', 'clusters', 'cell.type', 'multiplets', 'percent.mt'
    var: 'features'


In [88]:
full_data.obs['cell.type'].value_counts()

CD4 T cells          5089
CD14+ Monocytes      4183
FCGR3A+ Monocytes    1351
B cells              1324
NK cells              638
CD8 T cells           564
Dendritic cells       403
Megakaryocytes        120
Name: cell.type, dtype: int64

In [19]:
full_data.obs['cell_type']=full_data.obs['cell.type']

In [20]:
full_data.obs

,orig.ident,nCount_RNA,nFeature_RNA,label,tsne1,tsne2,ind,stim,clusters,cell.type,multiplets,percent.mt,cell_type
AAACATACATTTCC-1_1,SeuratProject,3018.0,878,control,-27.640373,14.966629,1016,ctrl,9,CD14+ Monocytes,singlet,0.099404,CD14+ Monocytes
AAACATACCAGAAA-1_1,SeuratProject,2481.0,713,control,-27.493646,28.924885,1256,ctrl,9,CD14+ Monocytes,singlet,0.241838,CD14+ Monocytes
AAACATACCTCGCT-1_1,SeuratProject,3420.0,850,control,-24.367997,20.429285,1256,ctrl,9,CD14+ Monocytes,singlet,0.087719,CD14+ Monocytes
AAACATACCTGGTA-1_1,SeuratProject,3158.0,1111,control,27.952170,24.159738,1039,ctrl,4,Dendritic cells,singlet,0.063331,Dendritic cells
AAACATACGATGAA-1_1,SeuratProject,1869.0,635,control,-0.470236,-25.398709,1488,ctrl,5,CD4 T cells,singlet,0.053505,CD4 T cells
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGCATGAACGAA-1_2,SeuratProject,5578.0,1238,case,-2.421106,28.893534,1244,stim,8,CD14+ Monocytes,singlet,0.089638,CD14+ Monocytes
TTTGCATGACGTAC-1_2,SeuratProject,1364.0,536,case,18.747179,1.919201,1488,stim,6,CD4 T cells,singlet,0.439883,CD4 T cells
TTTGCATGCCTGTC-1_2,SeuratProject,2116.0,819,case,-11.563067,2.574095,1256,stim,4,B cells,singlet,0.047259,B cells
TTTGCATGCTAAGC-1_2,SeuratProject,1522.0,523,case,25.142392,6.603815,107,stim,6,CD4 T cells,singlet,0.000000,CD4 T cells


In [21]:
full_data.X

<13672x35635 sparse matrix of type '<class 'numpy.float64'>'
	with 9530688 stored elements in Compressed Sparse Row format>

In [22]:
pathway_path= '/content/BMI_7320_Final/VEGA_PBMC/pathway/'

In [23]:
pathway_file_list=os.listdir(pathway_path)
pathway_file_list=[i for i in pathway_file_list if '.gmt' in i]

In [24]:
pathway_file_list

['GO_merged_non_redundant_full_term_name_gene_symbol_rename.gmt',
 'h.all.v2023.1.Hs.symbols.gmt']

##choose Hallmark (h.all.v2023.1.Hs.symbols.gmt) first and then run GO

In [25]:
pathway_file=pathway_file_list[1]

In [26]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['axes.linewidth'] = .5
matplotlib.rcParams['lines.linewidth'] = .5
matplotlib.rcParams['patch.linewidth'] = .5
matplotlib.rc('ytick', labelsize=14)
matplotlib.rc('xtick', labelsize=14)
import seaborn as sns


In [27]:
# Set model
random_seed = 1
torch.backends.cudnn.enabled = True
torch.manual_seed(random_seed)
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(dev)
LR = 1e-4
N_EPOCHS=20
p_drop = 0.5

dir_name='/content/BMI_7320_Final/VEGA_PBMC/save_model'


cuda


In [9]:
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cuda')

In [79]:
##use 'CD8 T cells' as an example
device = torch.device('cuda')
cell_type = 'CD8 T cells'
print('Training model leaving out '+cell_type, flush=True)
prefix_model = 'vega_kang_pbmc_%s_out_'%( cell_type)
print(prefix_model)
prefix_out = '5CV_vega_kang_pbmc_%s_out_res.npy'%(cell_type)
print(prefix_out)
cells = full_data.obs["cell_type"] ==cell_type
train_data = full_data[cells].copy()
labels = train_data.obs['cell_type']
le = preprocessing.LabelEncoder()
le.fit(labels)
y = torch.Tensor(le.transform(labels))
pathway_dict = read_gmt(pathway_path+pathway_file, min_g=0, max_g=1000)
pathway_mask = create_pathway_mask(train_data.var.index.tolist(), pathway_dict, add_missing=1, fully_connected=True)
print(pathway_mask)
if sparse.issparse(train_data.X):
  train_ds = train_data.X.A
else:
  train_ds = train_data.X
train_ds = torch.Tensor(train_ds).to(device)
print(1)
print(train_ds.is_cuda)
train_ds = UnsupervisedDataset(train_ds, targets=y)

# Initialize CV
kfold = KFoldTorch(cv=5, n_epochs=N_EPOCHS, lr=LR, train_p=10, test_p=10, num_workers=0, save_all=True, save_best=False, path_dir=dir_name, model_prefix=prefix_model)
dict_params = {'pathway_mask':pathway_mask, 'beta':0.00005, 'dropout':p_drop, 'path_model':None, 'device':dev, 'positive_decoder':True}
kfold.train_kfold(VEGA, dict_params, train_ds, batch_size=64)
np.save(dir_name+prefix_out, kfold.cv_res_dict)
print('Finished training for %s out'%(cell_type), flush=True)

Training model leaving out CD8 T cells
vega_kang_pbmc_CD8 T cells_out_
5CV_vega_kang_pbmc_CD8 T cells_out_res.npy
[[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
1
True
Model(s) will be saved at /content/BMI_7320_Final/VEGA_PBMC/save_model/ using vega_kang_pbmc_CD8 T cells_out_ as prefix
0.5
Constraining decoder to positive weights
cuda
Training fold 0
[Epoch 1] | loss: 79286.478 | test_loss: 30858.525 |
[Epoch 2] | loss: 62050.957 | test_loss: 31113.289 |
[Epoch 3] | loss: 61499.844 | test_loss: 31030.127 |
[Epoch 4] | loss: 61212.208 | test_loss: 37936.125 |
[Epoch 5] | loss: 61607.413 | test_loss: 29057.145 |
[Epoch 6] | loss: 61375.311 | test_loss: 26732.172 |
[Epoch 7] | loss: 60962.561 | test_loss: 30331.855 |
[Epoch 8] | loss: 61101.934 | test_loss: 31538.643 |
[Epoch 9] | loss: 61137.337 | test_loss: 31858.686 |
[Epoch 10] | loss: 61018.518 | test_loss: 36947.867 |
[Epoch

In [67]:
train_data.obs

,orig.ident,nCount_RNA,nFeature_RNA,label,tsne1,tsne2,ind,stim,clusters,cell.type,multiplets,percent.mt,cell_type
AAACATTGAGTGTC-1_1,SeuratProject,2157.0,882,control,-21.565959,-14.296156,1016,ctrl,4,CD8 T cells,singlet,0.324525,CD8 T cells
AAACCGTGTGCTAG-1_1,SeuratProject,1276.0,575,control,11.144007,-18.596850,1015,ctrl,10,CD8 T cells,singlet,0.235110,CD8 T cells
AAACGCTGACCTTT-1_1,SeuratProject,1040.0,569,control,-11.719508,-7.826960,1016,ctrl,3,CD8 T cells,singlet,0.192308,CD8 T cells
AAACGCTGGGTTTG-1_1,SeuratProject,1346.0,574,control,8.396368,-22.465294,1015,ctrl,10,CD8 T cells,singlet,0.222883,CD8 T cells
AAACGGCTGGTCAT-1_1,SeuratProject,1486.0,753,control,-11.577748,-7.834071,1016,ctrl,3,CD8 T cells,singlet,0.134590,CD8 T cells
...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTCGTATGTGCGTA-1_2,SeuratProject,1129.0,523,case,14.332360,-11.814588,101,stim,10,CD8 T cells,singlet,0.354296,CD8 T cells
TTCTAGTGTAGTCG-1_2,SeuratProject,1391.0,663,case,10.479714,-18.150953,1016,stim,10,CD8 T cells,singlet,0.215672,CD8 T cells
TTGCTATGTCTTTG-1_2,SeuratProject,1459.0,751,case,12.502294,-11.615107,1039,stim,10,CD8 T cells,singlet,0.342700,CD8 T cells
TTTAGGCTTCTTCA-1_2,SeuratProject,1135.0,505,case,28.889475,-11.562450,1016,stim,10,CD8 T cells,singlet,0.088106,CD8 T cells


In [80]:
##load the best model
device = torch.device('cuda')
model = VEGA(pathway_mask, positive_decoder=True)
# Load the best trained model weights
model.load_state_dict(torch.load('/content/BMI_7320_Final/VEGA_PBMC/save_model/vega_kang_pbmc_CD8 T cells_out_fold_4.pt',map_location=device))
model.to(device)
# Set the model to evaluation mode
model.eval()


0.2
Constraining decoder to positive weights


VEGA(
  (encoder): Sequential(
    (0): Linear(in_features=35635, out_features=800, bias=True)
    (1): BatchNorm1d(800, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=800, out_features=800, bias=True)
    (5): BatchNorm1d(800, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
  )
  (mean): Sequential(
    (0): Linear(in_features=800, out_features=51, bias=True)
    (1): Dropout(p=0.2, inplace=False)
  )
  (logvar): Sequential(
    (0): Linear(in_features=800, out_features=51, bias=True)
    (1): Dropout(p=0.2, inplace=False)
  )
  (decoder): CustomizedLinear(input_features=51, output_features=35635, bias=True)
)

In [81]:
next(model.parameters()).is_cuda

True

In [70]:
n = 1
pathway_list = list(pathway_dict.keys())+['UNANNOTATED_'+str(k) for k in range(n)]
print(pathway_list)

['HALLMARK_TNFA_SIGNALING_VIA_NFKB', 'HALLMARK_HYPOXIA', 'HALLMARK_CHOLESTEROL_HOMEOSTASIS', 'HALLMARK_MITOTIC_SPINDLE', 'HALLMARK_WNT_BETA_CATENIN_SIGNALING', 'HALLMARK_TGF_BETA_SIGNALING', 'HALLMARK_IL6_JAK_STAT3_SIGNALING', 'HALLMARK_DNA_REPAIR', 'HALLMARK_G2M_CHECKPOINT', 'HALLMARK_APOPTOSIS', 'HALLMARK_NOTCH_SIGNALING', 'HALLMARK_ADIPOGENESIS', 'HALLMARK_ESTROGEN_RESPONSE_EARLY', 'HALLMARK_ESTROGEN_RESPONSE_LATE', 'HALLMARK_ANDROGEN_RESPONSE', 'HALLMARK_MYOGENESIS', 'HALLMARK_PROTEIN_SECRETION', 'HALLMARK_INTERFERON_ALPHA_RESPONSE', 'HALLMARK_INTERFERON_GAMMA_RESPONSE', 'HALLMARK_APICAL_JUNCTION', 'HALLMARK_APICAL_SURFACE', 'HALLMARK_HEDGEHOG_SIGNALING', 'HALLMARK_COMPLEMENT', 'HALLMARK_UNFOLDED_PROTEIN_RESPONSE', 'HALLMARK_PI3K_AKT_MTOR_SIGNALING', 'HALLMARK_MTORC1_SIGNALING', 'HALLMARK_E2F_TARGETS', 'HALLMARK_MYC_TARGETS_V1', 'HALLMARK_MYC_TARGETS_V2', 'HALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION', 'HALLMARK_INFLAMMATORY_RESPONSE', 'HALLMARK_XENOBIOTIC_METABOLISM', 'HALLMARK_FATT

**Q1: fix the bug of cuda:0 and cpu inconsistency. This work needs GPU.**

In [86]:
z = model.to_latent(torch.Tensor(train_data.X.A)).detach().numpy()
pathway_encoded_df = pd.DataFrame(data=z, index=train_data.obs.index.tolist(), columns=pathway_list)


RuntimeError: ignored

In [85]:
reducer = umap.UMAP(random_state=42, min_dist=0.5, n_neighbors=15)
embedding = reducer.fit_transform(z)
plt.figure(figsize=[8,7])
plt.scatter(embedding[:,0], embedding[:,1], alpha = 1, linewidths=0,
            c = pathway_encoded_df['HALLMARK_INTERFERON_ALPHA_RESPONSE'], marker='o', s=10, cmap = 'seismic')
cbar = plt.colorbar()
for t in cbar.ax.get_yticklabels():
     t.set_fontsize(20)
plt.title('HALLMARK_INTERFERON_ALPHA_RESPONSE', fontsize=20)
plt.xlabel('UMAP-1', fontsize=20)
plt.ylabel('UMAP-2', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
#plt.savefig('../paper/figures/fig1-ifn-umap-deep.pdf', format='pdf', dpi=300, bbox_inches='tight')
plt.show()
plt.close()

NameError: ignored

**Q2 please reproduce the code on GO_merged_non_redundant_full_term_name_gene_symbol_rename.gmt and CD14+ Monocytes cells**

In [ ]:
full_data.obs['cell.type'].value_counts()

CD4 T cells          5089
CD14+ Monocytes      4183
FCGR3A+ Monocytes    1351
B cells              1324
NK cells              638
CD8 T cells           564
Dendritic cells       403
Megakaryocytes        120
Name: cell.type, dtype: int64

**Q3 run through these R code on Rstudio with preinstalled library**

In [31]:
##load R code for ssGSEA, following the tutorial https://bioconductor.org/packages/devel/bioc/vignettes/escape/inst/doc/vignette.html
%load_ext rpy2.ipython

In [32]:
##install ssGSEA pacakge
%%R

install.packages("Seurat")
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("escape")
BiocManager::install("dittoSeq")

suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(escape))
suppressPackageStartupMessages(library(dittoSeq))

(as ‘lib’ is unspecified)



























































































































































































































































































































































































	‘/tmp/RtmpyBlWBt/downloaded_packages’

(as ‘lib’ is unspecified)

Exception ignored from cffi callback <function _processevents at 0x7b2b837dcc10>:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/rpy2/rinterface_lib/callbacks.py", line 278, in _processevents
    processevents()
  File "/usr/local/lib/python3.10/dist-packages/rpy2/rinterface_lib/callbacks.py", line 262, in processevents
    def processevents() -> None:
KeyboardInterrupt: 






	‘/tmp/RtmpyBlWBt/downloaded_packages’



 

 

 

 

 

 

 

 

 

 




Error in loadNamespace(x) : there is no package called ‘BiocManager’


In [35]:
##load TP
%%R
GO_non_redundant <- read.gmt("/content/BMI_7320_Final/VEGA_PBMC/pathway/GO_merged_non_redundant_full_term_name_gene_symbol_rename.gmt")
hall_mark <- read.gmt("/content/BMI_7320_Final/VEGA_PBMC/pathway/h.all.v2023.1.Hs.symbols.gmt")

#convert to certain type
GO_non_redundant_unique_TP = as.vector(unique(GO_non_redundant[,1]))
GO_non_redundant_gene_sets = list()
for(i in 1:length(GO_non_redundant_unique_TP)) {
	GO_non_redundant_gene_sets = c(GO_non_redundant_gene_sets,list(GO_non_redundant[which(GO_non_redundant[,1]==GO_non_redundant_unique_TP[i]),2]))
}
names(GO_non_redundant_gene_sets) = GO_non_redundant_unique_TP

hall_mark_unique_TP = as.vector(unique(hall_mark[,1]))
hall_mark_gene_sets = list()
for(i in 1:length(hall_mark_unique_TP)) {
	hall_mark_gene_sets = c(hall_mark_gene_sets,list(hall_mark[which(hall_mark[,1]==hall_mark_unique_TP[i]),2]))
}
names(hall_mark_gene_sets) = hall_mark_unique_TP

  could not find function "read.gmt"




Error in read.gmt("/content/BMI_7320_Final/VEGA_PBMC/pathway/GO_merged_non_redundant_full_term_name_gene_symbol_rename.gmt") : 
  could not find function "read.gmt"


In [ ]:
##load object
%%R

load("/content/BMI_7320_Final/VEGA_PBMC/data/pbmc_IFN_filtered.Rdata")
seurat_object_merged_filtered = subset(seurat_object_merged_filtered, subset=cell.type == "CD8 T cells" )


In [ ]:
##ssgsea
hall_mark.ES.seurat <- enrichIt(obj = Homo.combined_regressed,
                      gene.sets = hall_mark_gene_sets,
                      groups = 1000, cores = 6)

GO_non_redundant.ES.seurat <- enrichIt(obj = Homo.combined_regressed,
                      gene.sets = GO_non_redundant_gene_sets,
                      groups = 1000, cores = 6)

In [ ]:
##visualize the heatmap
%%R

GO_non_redundant.ES.seurat_filter = GO_non_redundant.ES.seurat[,grep("interferon",colnames(GO_non_redundant.ES.seurat))]
p <- dittoHeatmap(seurat_object_merged_filtered, genes = NULL, metas = names(GO_non_redundant.ES.seurat_filter),
             annot.by = "stim",
             fontsize = 7,
             cluster_cols = TRUE)
print(p)

